In [1]:
import datetime
import time
import ccxt 
import pandas as pd 

def date_to_timestamp(date,utc=False):
    """
    str형태의 date를 timestamp로 만들어주기
    :params (str or datetime) date : '%Y-%m-%d %H:%M:%S'형태의 데이터. ex)'2023-01-18 23:00:00'
    :params bool utc : True로 설정할시에 date를 utc 시간이라고 생각
    :return timestamp시간(단위 ms) ex)1674050400000
    :rtype int
    
    ex) date_to_timestamp('2023-01-18 23:00:00') -> 1674050400000
    """
    if type(date) == str: # str인 경우 datetime으로 변환해주기
        dt = datetime.datetime.strptime(date,'%Y-%m-%d %H:%M:%S')
    else: # datetime으로 들어온경우
        dt = date
        
    # time.mktime은 local 타임 기준으로 timestamp를 변경함. 따라서 utc일 경우 +9시간해서 한국시간으로 설정해줘야함
    if utc:
        dt = date + datetime.timedelta(hours = 9)
    
    ts = time.mktime(dt.timetuple()) 
    
    return int(ts*1000)

def make_csv_data(coin_name,period,start_time,end_time):
    """
    coin이름, 수집하고 싶은 봉의 기준 기간, 시작, 끝 시간을 지정해주면 그 기간까지의 데이터를 수집하여 csv 파일로 반환
    :params str coin_name : 코인이름 ex)"BTC/USDT"
    :params str period : 수집기준기간 ex) "1m"
    :params str start_time : 수집 시작 시간 ex) '2022-01-01 00:00:00' (한국시간기준)
    :params str end_time : 수집 끝 시간 ex) '2023-01-01 00:00:00' (한국시간기준)
    :return None
    :rtype None
    
    f'{coin_name}_{period}_{start_time}_{end_time}.csv' 파일로 저장됨
    """
    
    # 입력 end_time은 한국시간기준이므로 utc기준으로 변경해주기 
    utc_end_time = datetime.datetime.strptime(end_time,'%Y-%m-%d %H:%M:%S') - datetime.timedelta(hours = 9)
    
    binance = ccxt.binanceusdm()
    btc_ohlcv = binance.fetch_ohlcv(coin_name,period,since=date_to_timestamp(start_time))

    df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
    df.set_index('datetime', inplace=True)

    while len(df)==0:  # 정기 점검이 있는 시간대에는 조회를 해도 결과가 나오지 않음. 500개씩 조회되므로 데이터가 조회될때까지 500개씩 건너뛰기
        if 'd' in period:
            start_time=datetime.datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(days=int(f"{period[:-1]}*500")) 
        elif 'm' in period:
            start_time=datetime.datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=int(f"{period[:-1]}*500")) 
        else:
            assert False, '일봉과 분봉만 조회 가능합니다.'

        btc_ohlcv = binance.fetch_ohlcv(coin_name,period,since=date_to_timestamp(start_time))

        df=pd.DataFrame(btc_ohlcv,columns=['datetime','open','high','low','close','volume'])
        df['datetime']=pd.to_datetime(df['datetime'],unit='ms')
        df.set_index('datetime',inplace=True)
    
    total_df = df
    
    check_count = 0
    
    while True:
        
        check_count+=1
        if check_count%10 == 0:
            print(total_df.index[-1])
        
        if utc_end_time <= df.index[-1]:
            break
        
        if 'd' in period:
            time_later=df.index[-1] + datetime.timedelta(days=int(f"{period[:-1]}")) 
        elif 'm' in period:
            time_later=df.index[-1] + datetime.timedelta(minutes=int(f"{period[:-1]}")) 
        else:
            assert False, '일봉과 분봉만 조회 가능합니다.'
        
        # pd.to_datetime는 utc기준으로 date를 반환함
        btc_ohlcv = binance.fetch_ohlcv(coin_name,period,since=date_to_timestamp(time_later,utc=True))

        df=pd.DataFrame(btc_ohlcv,columns=['datetime','open','high','low','close','volume'])
        df['datetime']=pd.to_datetime(df['datetime'],unit='ms')
        df.set_index('datetime',inplace=True)
        
        while len(df)==0:  # 정기 점검이 있는 시간대에는 조회를 해도 결과가 나오지 않음. 500개씩 조회되므로 데이터가 조회될때까지 500개씩 건너뛰기
            if 'd' in period:
                time_later=time_later + datetime.timedelta(days=int(f"{period[:-1]}*500")) 
            elif 'm' in period:
                time_later=time_later + datetime.timedelta(minutes=int(f"{period[:-1]}*500")) 
            else:
                assert False, '일봉과 분봉만 조회 가능합니다.'
            
            btc_ohlcv = binance.fetch_ohlcv(coin_name,period,since=date_to_timestamp(time_later,utc=True))

            df=pd.DataFrame(btc_ohlcv,columns=['datetime','open','high','low','close','volume'])
            df['datetime']=pd.to_datetime(df['datetime'],unit='ms')
            df.set_index('datetime',inplace=True)
            
        total_df = pd.concat([total_df,df])
        
    total_df.index = total_df.index + datetime.timedelta(hours = 9)
    total_df = total_df[:end_time]
    
    coin_name = "".join(coin_name.split("/"))
    s_time = "-".join("-".join(str(start_time).split(" ")).split(":"))
    e_time = "_".join("-".join(str(end_time).split(" ")).split(":"))
    total_df.to_csv(f'./{coin_name}_{period}_{s_time}_{e_time}.csv')

In [12]:
make_csv_data(coin_name= 'BTC/USDT', period= '1m', start_time= '2020-01-01 00:00:00', end_time= '2023-01-25 23:59:00')

2020-01-04 02:19:00
2020-01-07 13:39:00
2020-01-11 00:59:00
2020-01-14 12:19:00
2020-01-17 23:39:00
2020-01-21 10:59:00
2020-01-24 22:19:00
2020-01-28 09:39:00
2020-01-31 20:59:00
2020-02-04 08:19:00
2020-02-07 19:39:00
2020-02-11 06:59:00
2020-02-14 18:19:00
2020-02-18 05:39:00
2020-02-21 16:59:00
2020-02-25 04:19:00
2020-02-28 15:39:00
2020-03-03 02:59:00
2020-03-06 14:19:00
2020-03-10 01:39:00
2020-03-13 12:59:00
2020-03-17 00:19:00
2020-03-20 11:39:00
2020-03-23 22:59:00
2020-03-27 10:19:00
2020-03-30 21:39:00
2020-04-03 08:59:00
2020-04-06 20:19:00
2020-04-10 07:39:00
2020-04-13 18:59:00
2020-04-17 06:19:00
2020-04-20 17:39:00
2020-04-24 04:59:00
2020-04-27 16:19:00
2020-05-01 03:39:00
2020-05-04 14:59:00
2020-05-08 02:19:00
2020-05-11 13:39:00
2020-05-15 00:59:00
2020-05-18 12:19:00
2020-05-21 23:39:00
2020-05-25 10:59:00
2020-05-28 22:19:00
2020-06-01 09:39:00
2020-06-04 20:59:00
2020-06-08 08:19:00
2020-06-11 19:39:00
2020-06-15 06:59:00
2020-06-18 18:19:00
2020-06-22 05:39:00


## 1분봉 데이터 체크하기

In [2]:
import pandas as pd

file_name = './BTCUSDT_1m_2020-01-01-00-00-00_2023-01-25-23_59_00.csv'
df = pd.read_csv(file_name)

In [43]:
df[df['volume']==0]

,datetime,open,high,low,close,volume,next_datetime
390013,2020-09-27 20:13:00,10650.67,10650.67,10650.67,10650.67,0.0,2020-09-27 20:12:00
390014,2020-09-27 20:14:00,10650.67,10650.67,10650.67,10650.67,0.0,2020-09-27 20:13:00
614041,2021-03-02 10:01:00,49361.01,49361.01,49361.01,49361.01,0.0,2021-03-02 10:00:00
614042,2021-03-02 10:02:00,49361.01,49361.01,49361.01,49361.01,0.0,2021-03-02 10:01:00
614043,2021-03-02 10:03:00,49361.01,49361.01,49361.01,49361.01,0.0,2021-03-02 10:02:00
...,...,...,...,...,...,...,...
1265890,2022-05-29 02:10:00,28999.00,28999.00,28999.00,28999.00,0.0,2022-05-29 02:09:00
1265891,2022-05-29 02:11:00,28999.00,28999.00,28999.00,28999.00,0.0,2022-05-29 02:10:00
1265892,2022-05-29 02:12:00,28999.00,28999.00,28999.00,28999.00,0.0,2022-05-29 02:11:00
1265893,2022-05-29 02:13:00,28999.00,28999.00,28999.00,28999.00,0.0,2022-05-29 02:12:00


#### volume이 0인 부분이 125개 존재. 이부분은 백테스팅할때 제외시켜줘야함.
#### 2020-09-27 20:13:00	~	2020-09-27 20:14:00	
#### 2021-03-02 10:01:00   ~    2021-03-02 10:59:00
#### 2022-05-02 07:26:00   ~    2022-05-02 07:54:00	
#### 2022-05-29 01:40:00   ~   2022-05-29 02:14:00	

In [44]:
df['next_datetime']=df['datetime'].shift(1)

C:\Users\User\AppData\Local\Temp\ipykernel_46736\1521773945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['next_datetime']=df['datetime'].shift(1)


In [45]:
df

,datetime,open,high,low,close,volume,next_datetime
1,2020-01-01 00:01:00,7218.00,7225.00,7181.78,7208.00,1830.226,NaT
2,2020-01-01 00:02:00,7205.80,7222.00,7193.72,7221.92,950.621,2020-01-01 00:01:00
3,2020-01-01 00:03:00,7221.74,7256.08,7219.14,7229.00,1032.665,2020-01-01 00:02:00
4,2020-01-01 00:04:00,7229.00,7233.99,7210.00,7213.45,590.311,2020-01-01 00:03:00
5,2020-01-01 00:05:00,7215.20,7226.11,7202.45,7219.90,467.202,2020-01-01 00:04:00
...,...,...,...,...,...,...,...
1614235,2023-01-25 23:55:00,22474.50,22503.10,22467.40,22478.30,857.622,2023-01-25 23:54:00
1614236,2023-01-25 23:56:00,22478.30,22497.20,22467.40,22469.70,534.896,2023-01-25 23:55:00
1614237,2023-01-25 23:57:00,22469.70,22469.70,22435.00,22437.10,1495.762,2023-01-25 23:56:00
1614238,2023-01-25 23:58:00,22437.10,22459.50,22433.70,22449.40,745.578,2023-01-25 23:57:00


In [46]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')

C:\Users\User\AppData\Local\Temp\ipykernel_46736\1680795768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')


In [47]:
df['next_datetime'] = pd.to_datetime(df['next_datetime'], format='%Y-%m-%d %H:%M:%S')

C:\Users\User\AppData\Local\Temp\ipykernel_46736\3444218369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['next_datetime'] = pd.to_datetime(df['next_datetime'], format='%Y-%m-%d %H:%M:%S')


In [48]:
print(type(df.iloc[0]['datetime']))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [49]:
df

,datetime,open,high,low,close,volume,next_datetime
1,2020-01-01 00:01:00,7218.00,7225.00,7181.78,7208.00,1830.226,NaT
2,2020-01-01 00:02:00,7205.80,7222.00,7193.72,7221.92,950.621,2020-01-01 00:01:00
3,2020-01-01 00:03:00,7221.74,7256.08,7219.14,7229.00,1032.665,2020-01-01 00:02:00
4,2020-01-01 00:04:00,7229.00,7233.99,7210.00,7213.45,590.311,2020-01-01 00:03:00
5,2020-01-01 00:05:00,7215.20,7226.11,7202.45,7219.90,467.202,2020-01-01 00:04:00
...,...,...,...,...,...,...,...
1614235,2023-01-25 23:55:00,22474.50,22503.10,22467.40,22478.30,857.622,2023-01-25 23:54:00
1614236,2023-01-25 23:56:00,22478.30,22497.20,22467.40,22469.70,534.896,2023-01-25 23:55:00
1614237,2023-01-25 23:57:00,22469.70,22469.70,22435.00,22437.10,1495.762,2023-01-25 23:56:00
1614238,2023-01-25 23:58:00,22437.10,22459.50,22433.70,22449.40,745.578,2023-01-25 23:57:00


In [50]:
## 1분봉은 연속되지 않은 데이터는 없음 무조건 전부 연속적인 데이터

import datetime
df[df['datetime']!=df['next_datetime']+datetime.timedelta(minutes = 1)]

,datetime,open,high,low,close,volume,next_datetime
1,2020-01-01 00:01:00,7218.0,7225.0,7181.78,7208.0,1830.226,NaT
